# Requirements and Imports

In [1]:
%pip install --quiet -r requirements.txt


[notice] A new release of pip is available: 23.3 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


# Setting up the experiment

In [2]:
# TEMP WIP
import matplotlib.pyplot as plt
import seaborn as sns
from metrics import np_quadratic_weighted_kappa, from_obdout_to_labels
from sklearn.metrics import confusion_matrix

In [3]:
from experiment import Experiment

# =============== Settings ===============
seed = 42
exps = 'exps/experiments.json'
exp_idx = 0

# ============== Experiment ==============
experiment = Experiment(seed, exps, exp_idx)
experiment.build()

[experiment] GPU acceleration available.
[dataset] 277 videos (58924 frames) loaded from cached data.


## Dataset Splitting

In [4]:
experiment.split_dataset()

[dataset] dataset split: train=37182(63%), val=10010(17%), test=11732(20%)
[dataset] dataset reduction: 15% (train=5577, val=1501, test=1759)


### Dataset Charts

In [5]:
experiment.plot_split_charts([0, 0, 0])

# Neural Network Model

In [6]:
model = experiment.nn_model()

2023-10-26 23:09:18.274751: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2023-10-26 23:09:18.274773: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2023-10-26 23:09:18.274776: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2023-10-26 23:09:18.274966: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-10-26 23:09:18.274982: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [7]:
experiment.nn_model_compile(model)

KeyError: 'loss'

# Training Neural Network

In [ ]:
history = experiment.nn_model_train(model)

In [ ]:
# Plot Training loss
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.legend()
plt.xlabel('epoch')
plt.ylabel('QWK loss')
plt.title('ResNet18-CLM for Ordinal Classification')
plt.grid()
plt.show

In [ ]:
# Plot Training accuracy
plt.plot(history.history['ccr'], label='train accuracy')
plt.plot(history.history['val_ccr'], label='val accuracy')
plt.legend()
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.title('ResNet18-CLM for Ordinal Classification')
plt.grid()
plt.show

# Evaluate

In [ ]:
model.evaluate(experiment.test_ds)

In [ ]:
y_test_pred = model.predict(experiment.test_ds)
# y_test_pred = np.argmax(y_test_pred, axis=1) # extract classes from probabilities

In [ ]:
y_test_pred = from_obdout_to_labels(y_test_pred)

In [ ]:
# Test Set Confusion Matrix
cf_matrix_test = confusion_matrix(y_test_ds, y_test_pred, normalize='true', labels=list(range(experiment.ds_num_classes)))

ax = sns.heatmap(cf_matrix_test, linewidths=1, annot=True, fmt='.2f')
ax.set_ylabel(f'Actual')
ax.set_xlabel(f'Predicted')
ax.set_title(f'Test Set Confusion Matrix')
plt.show()

In [ ]:
qwk_test = np_quadratic_weighted_kappa(y_test_ds, y_test_pred, 0, experiment.ds_num_classes - 1)

print('QWK Test Set: {:.4f}'.format(qwk_test))